# Imports and Setup


In [1]:
import os, sys, torch, cv2, numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pathlib import Path
import pandas as pd

os.chdir('/Users/devaguru/Kidney Stone CNN/kidney-stone-cnn')
sys.path.insert(0, '.')

from src.evaluation.gradcam import load_model, generate_gradcam

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model  = load_model('checkpoints/best_model.pth', device)
print('Model loaded on:', device)



Model loaded on: mps


# Generate Grad-CAM grid for stone images

In [ ]:
df = pd.read_csv('data/labels/splits.csv')
stone_test = df[(df.split=='test') & (df.label=='stone')].head(10)

fig, axes = plt.subplots(2, 10, figsize=(30, 6))
fig.suptitle('Grad-CAM — Stone Images (Test Set)', fontsize=14, fontweight='bold')

for j, (_, row) in enumerate(stone_test.iterrows()):
    img_path = f"data/processed/test/stone/{row['filename']}"
    result   = generate_gradcam(model, img_path, device, target_class=1)

    # Top row: original image
    orig = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    axes[0][j].imshow(orig)
    axes[0][j].set_title(f"True: stone", fontsize=7)
    axes[0][j].axis('off')

    # Bottom row: Grad-CAM overlay
    axes[1][j].imshow(result['overlay'])
    color = 'green' if result['prediction'] == 'stone' else 'red'
    axes[1][j].set_title(
        f"Pred: {result['prediction']}\n{result['confidence']:.2%}",
        fontsize=7, color=color)
    axes[1][j].axis('off')
plt.tight_layout()
plt.savefig('reports/gradcam_stone.png', dpi=120, bbox_inches='tight')
plt.show()
print('Saved to reports/gradcam_stone.png')
